In [1]:
import json, requests, time, datetime

with open('/home/gxk/Downloads/www.bluenile.com.har') as fr:
    har_js = json.load(fr)

coll = []
for xhr in har_js['log']['entries']:
    if xhr['request']['url'].find('startIndex')>-1:
        xhr_js = json.loads(xhr['response']['content']['text'])
        coll.extend(xhr_js['results']) 
        
import pandas as pd

pd.set_option('display.max_columns', None)
df = pd.DataFrame(coll)
df = df.rename(columns={'shapeName':'Shape','dateSet':'Delivery Date'})
df = df[['Shape','price','carat','cut','color','clarity','Delivery Date']]
df['Shape'] = df.apply(lambda x: x.Shape[0], axis=1)
df['price'] = df.apply(lambda x: x.price[0], axis=1)
df['carat'] = df.apply(lambda x: x.carat[0], axis=1)
df['cut'] = df.apply(lambda x: x.cut[0]['label'], axis=1)
df['color'] = df.apply(lambda x: x.color[0], axis=1)
df['clarity'] = df.apply(lambda x: x.clarity[0], axis=1)
df['Delivery Date'] = df.apply(lambda x: x['Delivery Date'][0], axis=1)

df

,Shape,price,carat,cut,color,clarity,Delivery Date
0,Round,$349,0.32,Ideal,K,SI2,Aug 6
1,Round,$349,0.30,Very Good,K,SI1,Aug 4
2,Round,$349,0.27,Ideal,H,SI2,Aug 4
3,Round,$350,0.35,Ideal,K,SI2,Aug 3
4,Round,$350,0.23,Good,H,IF,Aug 4
...,...,...,...,...,...,...,...
145,Round,$357,0.30,Ideal,K,SI2,Aug 6
146,Round,$358,0.31,Ideal,J,SI2,Aug 5
147,Round,$358,0.30,Ideal,J,SI2,Aug 6
148,Round,$358,0.37,Very Good,K,SI2,Aug 4


In [2]:
from urllib.parse import urlparse, parse_qs, parse_qsl, urlencode 

for xhr in har_js['log']['entries']:
    if xhr['request']['url'].find('startIndex')>-1:
        xhr_js = json.loads(xhr['response']['content']['text'])
        coll.extend(xhr_js['results']) 
        break
o = urlparse(xhr['request']['url'])
query = parse_qs(o.query) 
query = dict(parse_qsl(o.query)) 
query.update({'_':f'{time.time():.3f}'.replace('.','')})
#query.update({'startIndex':'111'}) 
url = f'{o.scheme}://{o.hostname}{o.path}?{urlencode(query)}' 
headers = {i['name']:i['value'] for i in xhr['request']['headers'] if i['name'][0]!=':'}
r = requests.get(url, headers=headers)
print(r.status_code)
r.json()

200


{'startIndex': 710,
 'endIndex': 759,
 'countRaw': 312671,
 'recommendedCountRaw': 20,
 'count': '312,671',
 'sortColumn': 'default',
 'sortDirection': 'asc',
 'results': [{'detailsPageUrl': './diamond-details/LD16360958',
   'carat': ['0.32'],
   'date': ['Aug 5'],
   'dateSet': ['Aug 6'],
   'price': ['$349'],
   'strikethroughPrice': None,
   'pricePerCarat': ['$1,091'],
   'skus': ['LD16360958'],
   'id': 'LD16360958',
   'shapeCode': ['RD'],
   'shapeName': ['Round'],
   'clarity': ['SI2'],
   'color': ['K'],
   'culet': ['None'],
   'cut': [{'label': 'Ideal', 'labelSmall': 'Ideal'}],
   'depth': ['62.6'],
   'fluorescence': ['Faint'],
   'lxwRatio': ['1.01'],
   'polish': ['Excellent'],
   'symmetry': ['Excellent'],
   'table': ['58.0'],
   'hasVisualization': [False],
   'visualizationImageUrl': None,
   'v360BaseUrl': None,
   'measurements': [{'label': '4.38 x 4.34 x 2.73 mm',
     'labelSmall': '4.38 x 4.34 x 2.73'}],
   'willArriveForHoliday': False,
   'imageUrl': None,
   

In [3]:
url

'https://www.bluenile.com/api/public/diamond-search-grid/v2?startIndex=710&pageSize=50&_=1626655777323&unlimitedPaging=false&sortDirection=asc&sortColumn=default&shape=RD&maxDateType=MANUFACTURING_REQUIRED&isQuickShip=false&hasVisualization=false&isFiltersExpanded=false&astorFilterActive=false&country=USA&language=en-us&currency=USD&productSet=BN'

In [ ]:
# jar = requests.cookies.RequestsCookieJar()
# _ = [jar.update(i) for i in xhr['request']['cookies']]
# jar